<font size='5'><b>GROUP BY & AGGREGATE FUNCTIONS</b><font>

In [4]:
import pandas as pd
import sqlite3
connect = sqlite3.Connection('data.sqlite')
cur = connect.cursor()

![Figure_1](img/SQL_Schema.png)

Let's start by looking at some GROUP BY statements to aggregate our data. The GROUP BY clause groups records into summary rows and returns one record for each group. Typically, GROUP BY also involves an aggregate function (COUNT, AVG, etc.). Lastly, GROUP BY can group by one or more columns.

In the cell below, we'll join the offices and employees tables in order to count the number of employees per city.

In [6]:
cur.execute("""SELECT city, COUNT(employeeNumber)
               FROM offices
               JOIN employees
               USING (officeCode)
               GROUP by city
               ORDER BY count(employeeNumber) DESC;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,city,COUNT(employeeNumber)
0,San Francisco,6
1,Paris,5
2,Sydney,4
3,Boston,2
4,London,2


<b>Aliasing</b>

An Alias is a shorthand for a table or column name. Aliases reduce the amount of typing required to enter a query. Generally, complex queries with aliases are easier to read. Aliases are useful with JOIN, GROUP BY, and aggregates (SUM, COUNT, etc.). An Alias only exists for the duration of the query.

You can alias your GROUP BY by specifying the index of our selection order that we want to group by. This is simply written as GROUP BY 1, with the number "1" referring to the first column name that we are selecting.

Additionally, we can also rename our aggregate to a more descriptive name using the AS clause.

In [8]:
cur.execute("""SELECT city, COUNT(employeeNumber) AS numEmployees
               FROM offices
               JOIN employees
               USING(officeCode)
               GROUP BY 1
               ORDER BY numEmployees DESC;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,city,numEmployees
0,San Francisco,6
1,Paris,5
2,Sydney,4
3,Boston,2
4,London,2


<b>Other Aggregations</b>

Aside from COUNT() some other useful aggregations include:

- MIN()
- MAX()
- SUM()
- AVG()

In [11]:
cur.execute("""SELECT customerName,
               COUNT(customerName) AS number_purchases,
               MIN(amount) AS min_purchase,
               MAX(amount) AS max_purchase,
               AVG(amount) AS avg_purchase,
               SUM(amount) AS total_spent
               FROM customers
               JOIN payments
               USING(customerNumber)
               GROUP BY customerName
               ORDER BY SUM(amount) DESC;""")
df = pd.DataFrame(cur.fetchall())
df. columns = [x[0] for x in cur.description]
print("df length is : {}".format(len(df)))
df.head()

df length is : 98


,customerName,number_purchases,min_purchase,max_purchase,avg_purchase,total_spent
0,Euro+ Shopping Channel,13,116208.40,65071.26,55056.844615,715738.98
1,Mini Gifts Distributors Ltd.,9,101244.59,85410.87,64909.804444,584188.24
2,"Australian Collectors, Co.",4,44894.74,82261.22,45146.267500,180585.07
3,Muscle Machine Inc,4,20314.44,58841.35,44478.487500,177913.95
4,"Dragon Souveniers, Ltd.",4,105743.00,44380.15,39062.757500,156251.03


<b>The HAVING clause</b>

Finally, we can also filter our aggregated views with the HAVING clause. The HAVING clause works similarly to the WHERE clause, except it is used to filter data selections on conditions after the GROUP BY clause. For example, if we wanted to filter based on a customer's last name, we would use the WHERE clause. However, if we wanted to filter a list of cities with at least 5 customers, we would use the HAVING clause. First, we would GROUP BY city and then use the HAVING clause, which will allow us to pass conditions on the result of this aggregation.

In [12]:
cur.execute("""SELECT city, COUNT(customerNumber) AS number_customers
               FROM customers
               GROUP BY 1
               HAVING COUNT(customerNumber)>=5;""")
df = pd.DataFrame(cur.fetchall())
df. columns = [x[0] for x in cur.description]
print(len(df))
df.head()

2


,city,number_customers
0,Madrid,5
1,NYC,5


<b>Combining the WHERE and HAVING clause</b>

We can also use the WHERE and HAVING clauses in conjunction with each other for more complex rules. For example, let's say we want a list of customers who have made at least 2 purchases of over 50K each.



In [13]:
cur.execute("""SELECT customerName,
               COUNT(amount) AS number_purchases_over_50K
               FROM customers
               JOIN payments
               USING(customerNumber)
               WHERE amount >= 50000
               GROUP BY customerName
               HAVING count(amount) >= 2
               ORDER BY count(amount) DESC;""")
df = pd.DataFrame(cur.fetchall())
df. columns = [i[0] for i in cur.description]
print(len(df))
df.head()

94


,customerName,number_purchases_over_50K
0,Euro+ Shopping Channel,13
1,Mini Gifts Distributors Ltd.,9
2,"Anna's Decorations, Ltd",4
3,"Australian Collectors, Co.",4
4,Baane Mini Imports,4


===================================================================

In [16]:
conn = sqlite3.connect('babe_ruth.db')
cur = conn.cursor()

We will query from a table containing data of Babe Ruth statistics that has the structure below. The name of the table is babe_ruth_stats, it can be found in the babe_ruth.db file in this repo.

<b>Total Seasons</b>

In [19]:
cur.execute("""SELECT COUNT(year) AS total_seasons
               FROM babe_ruth_stats;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,total_seasons
0,22


<b>Seasons with NY</b>

In [23]:
cur.execute("""SELECT COUNT(year) as NY_seasons
               FROM babe_ruth_stats
               WHERE team = "NY";""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,NY_seasons
0,15


<b>Most HR in 1 season</b>

In [24]:
cur.execute("""SELECT * 
               FROM babe_ruth_stats
               ORDER BY HR DESC
               LIMIT 1;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,id,year,team,league,doubles,triples,hits,HR,games,runs,RBI,at_bats,BB,SB,SO,AVG
0,14,1927,NY,AL,29,8,192,60,151,158,164,540,137,7,89,0.356


<b>Least HR in a season</b>

In [26]:
cur.execute("""SELECT * 
               FROM babe_ruth_stats
               ORDER BY HR ASC
               LIMIT 1;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,id,year,team,league,doubles,triples,hits,HR,games,runs,RBI,at_bats,BB,SB,SO,AVG
0,1,1914,BOS,AL,1,0,2,0,5,1,2,10,0,0,4,0.2


<b>Total HR</b>

In [29]:
cur.execute("""SELECT sum(HR) AS total_HR
               FROM babe_ruth_stats;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,total_HR
0,714


<b>Five Worst HR Seasons With at Least 100 Games Played</b>

In [30]:
cur.execute("""SELECT * 
               FROM babe_ruth_stats
               WHERE games >= 100
               ORDER BY HR ASC
               LIMIT 5;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,id,year,team,league,doubles,triples,hits,HR,games,runs,RBI,at_bats,BB,SB,SO,AVG
0,21,1934,NY,AL,17,4,105,22,125,78,84,365,104,1,63,0.288
1,6,1919,BOS,AL,34,12,139,29,130,103,114,432,101,7,58,0.322
2,20,1933,NY,AL,21,3,138,34,137,97,103,459,114,4,90,0.301
3,9,1922,NY,AL,24,8,128,35,110,94,99,406,84,2,80,0.315
4,10,1923,NY,AL,45,13,205,41,152,151,131,522,170,17,93,0.393


<b>Average Batting Average</b>

In [33]:
cur.execute("""SELECT AVG(AVG) AS career_avg
               FROM babe_ruth_stats;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,career_avg
0,0.322864


<b>Total Years and Hits Per Team</b>

In [36]:
cur.execute("""SELECT team, COUNT(year) AS number_years, SUM(hits) AS number_hits
               FROM babe_ruth_stats
               GROUP BY team;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,team,number_years,number_hits
0,BOS,7,355
1,NY,15,2518


<b>Number of Years with Over 300 Times on Base</b>

In [38]:
cur.execute("""SELECT year, hits + BB AS on_base
               FROM babe_ruth_stats
               GROUP BY year
               HAVING on_base > 300
               ORDER BY on_base DESC;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head(10)

,year,on_base
0,1923,375
1,1921,349
2,1924,342
3,1927,329
4,1926,328
5,1931,327
6,1920,322
7,1930,322
8,1928,310
